In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt
import os

In [11]:

def obtener_subbandas_diagonales(imagen_ycbcr, wavelet='db1'):
    """
    Aplica la transformada wavelet a cada canal de la imagen YCbCr y extrae las subbandas diagonales (HH).

    :param imagen_ycbcr: Imagen en formato YCbCr (NumPy array).
    :return: Lista con las subbandas diagonales (HH) de los canales [Y_HH, Cb_HH, Cr_HH].
    """
    subbandas_HH = []
    canales = cv2.split(imagen_ycbcr)  # Separar los canales Y, Cb, Cr 

    for canal in canales:               #Recorro los canales 
        coeficientes = pywt.wavedec2(canal, wavelet=wavelet, level=1)# Wavedec2 descompone la imagen en coeficientes de aproximación (LL), horizontal(HL), vertical(LH) y diagonal (HH)
        _, *detalles = coeficientes # Extraer la subbanda diagonal (HH) en el nivel deseado
        HH = detalles[0][2]  # Accedemos a la subbanda HH en el nivel seleccionado
        subbandas_HH.append(HH)
    
    return subbandas_HH


In [34]:
def calcular_matrices_T(imagen_ycbcr, alpha=1, block_size=8):
    """
    Calcula las matrices TY, TCb y TCr utilizando la ecuación dada.
    :param imagen_ycbcr: Imagen original en YCbCr.
    :param alpha: Parámetro de ponderación.
    :param block_size: Tamaño de bloque para el cálculo de la media y desviación estándar local.
    :return: Matrices TY, TCb, TCr.
    """
    MH = []  # Matrices de alta frecuencia ajustadas
    S = []   # Matrices de desviación estándar
    T = []   # Matrices de salida TY, TCb, TCr
    HH_subbandas = obtener_subbandas_diagonales(imagen_ycbcr, wavelet='db1')

    # Separar los canales de la imagen
    canales = cv2.split(imagen_ycbcr)
    
    for i in range(3):
        H = HH_subbandas[i]
        canal = canales[i]
        
        # Calcular la media de bloques no superpuestos
        mean_filter = np.ones((block_size, block_size)) / (block_size ** 2)
        mean_H = cv2.filter2D(H, -1, mean_filter)
        MH_i = np.abs(H - mean_H)                                              #La matriz tiene menos tamaño que la imagen original ya que se divide en bloques
        MH.append(MH_i)
        
        # Calcular la desviación estándar local
        mean_canal = cv2.filter2D(canal.astype(float), -1, mean_filter)
        std_canal = np.sqrt(cv2.filter2D((canal - mean_canal) ** 2, -1, mean_filter))
        std_canal = cv2.resize(std_canal, (MH_i.shape[1], MH_i.shape[0]), interpolation=cv2.INTER_NEAREST) #Ajusto el tamaño 

        S.append(std_canal)
       
        # Calcular la matriz T según la ecuación del árticulo científico
        numerador = (MH_i ** alpha) * std_canal
        denominador = np.sum(std_canal) if np.sum(std_canal) != 0 else 1
        T_i = numerador / denominador
        T.append(T_i)
        
    
    return T[0], T[1], T[2]  # TY, TCb, TCr



In [38]:
def calcular_estimulo_total(TY, TCb, TCr, alpha=1):
    """
    Calcula el total stimulus T_S de la imagen usando la ecuación dada.
    """
    suma = (TY + TCb + TCr) / 3     # Sumar las matrices de los tres canales
    T_S = np.power(suma, 1/alpha)   # Calcular el total stimulus T_S
    
    return T_S

In [ ]:
def calcular_Smap(T_S, epsilon=1e-6):
    """
    Calcula la matriz Smap (mapa de nitidez) utilizando la ecuación dada:
    Smap(i, j) = abs[log(ε) + ε] / (abs[log(T_S(i, j)) + ε] + ε)
    """
    # Numerador: abs(log(epsilon) + epsilon)
    numerador = np.abs(np.log(epsilon) + epsilon)
    denominador = np.abs(np.log(T_S + epsilon)+ epsilon) # Denominador: abs(log(T_S) + epsilon) + epsilon
    # Calcular Smap
    Smap = numerador / denominador
    return Smap

In [ ]:
def obtener_BSmap(Smap, block_size=8):
    """
    Elimina el borde del mapa de nitidez (Smap) en función del tamaño del bloque.
    """
    # El tamaño del borde a eliminar es block_size - 1 en cada dirección
    border_size = block_size - 1
    # Recortar el borde del Smap
    BSmap = Smap[border_size:-border_size, border_size:-border_size] if border_size > 0 else Smap

    return BSmap

In [46]:
# Ejemplo de uso
imagen_ycbcr = cv2.imread("000002_sharp.png") # Cargar imagen YCbCr
TY, TCb, TCr = calcular_matrices_T(imagen_ycbcr)
T_S = calcular_estimulo_total(TY, TCb, TCr)
Smap = calcular_Smap(T_S)
print(Smap)


[[1.00370157 1.09292556 1.00270774 ... 1.00970469 1.00852727 1.0058809 ]
 [1.00396148 1.00413238 1.10243359 ... 1.00946396 1.00836568 1.00572285]
 [1.00544387 1.09343891 1.0028022  ... 1.00789202 1.00622379 1.00388402]
 ...
 [1.43818915 1.2560771  1.34242874 ... 1.07573879 1.00700196 1.00662294]
 [1.1262043  1.12269198 1.27804786 ... 1.10788049 1.0413621  1.09543219]
 [1.15021191 1.14769333 1.03416703 ... 1.046097   1.04440633 1.01557293]]


In [ ]:

# Cargar imagen en BGR y convertir a YCbCr
imagen_bgr = cv2.imread("000002_sharp.png")
imagen_ycbcr = cv2.cvtColor(imagen_bgr, cv2.COLOR_BGR2YCrCb)

# Obtener las subbandas diagonales
Y_HH, Cb_HH, Cr_HH = obtener_subbandas_diagonales(imagen_ycbcr)
# Mostrar las subbandas diagonales
cv2.imshow("Y_HH", np.uint8(np.abs(Y_HH)))
cv2.imshow("Cb_HH", np.uint8(np.abs(Cb_HH)))
cv2.imshow("Cr_HH", np.uint8(np.abs(Cr_HH)))
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 0.00000000e+00 -5.00000000e-01  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.00000000e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  5.00000000e-01  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 5.00000000e-01 -5.00000000e-01 -1.50000000e+00 ... -5.00000000e-01
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  5.00000000e-01 ...  5.00000000e-01
  -5.00000000e-01 -5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  5.00000000e-01
   1.01030295e-14  0.00000000e+00]]


 ### 🧮 Calculo transformada de wavelet

Esta función `wavelet_alta_frecuancia` aplica la transformada de wavelet a una imagen en escala de grises. El objetivo es poder obtener los coeficientes de alta frecuencia horizontal, vertical y diagonal 

**¿Cómo funciona?**
- Primero calcula la transformada de wavelet de la imagen con `pywt.wavedec2()`.
- Luego, en el resultado de la transformada me quedo solo con los coeficientes horizontal, vertical y diagonal, es decir, descarto el coeficiente de aproximación, ya que este devuelve baja frecuencia.

Este tipo de filtro es útil para análisis de nitidez, detección de bordes o estudios de textura.

In [ ]:
def wavelet_subbandas_diagonales(img_YCbCr, wavelet='db1'):
    """
    Aplica la Transformada Wavelet Discreta 2D a la imagen y devuelve los coeficientes de alta frecuencia.
    """
    coeficientes = pywt.wavedec2(img_gray, wavelet=wavelet, level=1) #wavedec2 descompone la imagen en coeficientes de alta frecuencia; level indica qeu solo es una descomposición
    coefs_alta_frecuencia = coeficientes[1:]  # Ignoramos el coeficiente de aproximación, ya que este devuelve baja frecuencia y no nos interesa
    return coefs_alta_frecuencia # Devuelve coeficientes -->  [(horizontal), (vertical), (diagonal)]

In [ ]:
def valor_nitidez_basado_coeficientes(coefs_alta_frecuencia):
    """
    Calcula una métrica de nitidez basada en la media de los coeficientes de alta frecuencia.
    """
    n_coef = 0
    total_coef = 0

    for i in coefs_alta_frecuencia:          # Recorro ceoficiente horizontal (LH), vertical (HL) y diagonal (HH)
        for j in i:                          # Recorro la matriz de cada nivel 
            n_coef += j.size                 # Cuetno nº total de coeficientes en dicha matriz
            total_coef += np.sum(np.abs(j))  # Sumo los coeficientes en absoluto

    if n_coef == 0:                          #No divido por 0 en la media
        return 0

    return total_coef / n_coef               # Media

### 🗂️ Procesamiento masivo de imágenes con wavelet

Esta función `procesar_imagenes_carpeta_wavelet` aplica la transformada de wavelet para calcular los coeficientes de alta frecuencia horizontal, vertical y diagonal y calcula la media de estos coeficientes en todas la imagenes `.png` dentro de una estructura de carpetas. 

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Genera el coeficiente vertical, horizontal y diagonal a cada imagen.
- Calcula el valor medio de estos coeficientes. 
- Guarda las imágenes resultantes (horizontal, vertical y diagonal) respetando la misma estructura de carpetas.
- Guarda la media de los coeficientes de cada imagen en un archivo `.txt`.

**Utilidad:**
Automatiza el análisis de contenido de alta frecuencia usando la transformada de wavelet, un método con mucha precisión ya que obtiene el contenido de alta frecuencia de tres fuentes (horizontal, vertical y diagonal)

In [ ]:
def procesar_imagenes_carpeta_wavelet(
    carpeta_entrada="images",
    carpeta_salida="images_high_frec",
    wavelet='db1' #Tipo de wavelet
):
    """
    Recorre todas las subcarpetas dentro de 'carpeta_entrada'.
    - Para cada archivo .png, aplica la transformada de wavelet
    - Guarda la imagen resultante en 'carpeta_salida', manteniendo la misma estructura.
    - Genera un archivo .txt con el valor de nitidez de cada imagen procesada.

    Parámetros:
    - carpeta_entrada: ruta de la carpeta de entrada.
    - carpeta_salida: ruta de la carpeta donde se guardarán los resultados.
    - wavelet: especifica el tipo de wavelet que se aplica, por defecto Daubechies de 1 nivel.
    
    """
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)
        # Añadimos la subcarpeta específica para 'wavelet'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "wavelet")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
        
        # Creamos un archivo TXT para guardar las varianzas
        ruta_txt = os.path.join(carpeta_salida_sub, "info_nitidez_wavelet.txt")
        
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Coeficientes de alta frecuencia (Transformada de wavelet)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("Valores mayores suelen corresponder a más bordes y mayor nitidez.\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if filename.lower().endswith(".png"):
                    ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                    
                    # Cargar imagen en YCbCr
                    img_bgr = cv2.imread(ruta_imagen_entrada)
                    if img_bgr is None:
                        continue
                    img_YCbCr = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
                    
                    # Obtengo mapa de alta frecuencia usando transformada de wavelet 
                    coeficientes_alta_frecuencia = wavelet_alta_frecuancia(img_YCbCr, wavelet=wavelet)
                    
                    ####CAMBIAR A PARTIR DE AQUI

                    
                    # Calcular valor medio de los coeficientes 
            
                    coef_medio = valor_nitidez_basado_coeficientes(coeficientes_alta_frecuencia) 
                    
                    # Guardar imágenes de los coeficientes de alta frecuencia

                    nombre_salida = os.path.splitext(filename)[0]  # Nombre imagen sin .png
                    nombre_imagen = ['LH_Horizontal', 'HL_Vertical Detail', 'HH_Diagonal Detail']
                    for i, coef in enumerate(coeficientes_alta_frecuencia[0]): #i = indice; coef = matrices LH, HL, HH

                        coef_frecuencias_escalada = np.abs(coef).astype(np.uint8)   # Crear una nueva imagen para cada coeficiente
                                                                                    # Construir ruta de salida para cada coeficiente
                        ruta_imagen_salida = os.path.join(carpeta_salida_sub, f"{nombre_salida}_{nombre_imagen[i].replace(' ', '_')}.png")
                        
                        # Guardar imagen del coeficiente
                        cv2.imwrite(ruta_imagen_salida, coef_frecuencias_escalada)

                    # Guardar resultado en el TXT
                    archivo_txt.write(f"{filename} -> Coeficiente medio: {coef_medio:.3f}\n")

    print("Procesamiento completado con transformada de wavelet.")

In [ ]:
# Ejecuta el procesamiento
procesar_imagenes_carpeta_wavelet(
    carpeta_entrada="imagenes",
    carpeta_salida="imagenes_procesadas",
)